In [4]:
pip install timm

     ---------------------------------------- 0.0/549.1 kB ? eta -:--:--
     ---- ---------------------------------- 61.4/549.1 kB 1.7 MB/s eta 0:00:01
     -------- ----------------------------- 122.9/549.1 kB 1.4 MB/s eta 0:00:01
     ---------------- --------------------- 235.5/549.1 kB 1.8 MB/s eta 0:00:01
     ---------------- --------------------- 235.5/549.1 kB 1.8 MB/s eta 0:00:01
     ----------------- -------------------- 256.0/549.1 kB 1.1 MB/s eta 0:00:01
     ------------------------- ------------ 368.6/549.1 kB 1.3 MB/s eta 0:00:01
     ------------------------------- ------ 450.6/549.1 kB 1.5 MB/s eta 0:00:01
     ------------------------------------ - 532.5/549.1 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 549.1/549.1 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: C:\Users\KIIT\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests

url = "WhatsApp Image 2023-04-29 at 12.46.16.jpeg"
# image = Image.open(requests.get(url, stream=True).raw)
image=Image.open("WhatsApp Image 2023-04-29 at 12.46.16.jpeg")

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

objects_detected=[]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
#     print(
#             f"Detected {model.config.id2label[label.item()]} with confidence "
#             f"{round(score.item(), 3)} at location {box}"
#     )
    objects_detected.append(model.config.id2label[label.item()])
objects_detected    

[]

In [5]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

model_name = "bipin/image-caption-generator"

# load model
model = VisionEncoderDecoderModel.from_pretrained(model_name)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KIIT\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\KIIT\AppData\Local\Packages\PythonSoftwareFoun

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, 

In [24]:
img_name = "Toothbrush.jpeg"
img = Image.open(img_name)
if img.mode != 'RGB':
    img = img.convert(mode="RGB")

In [25]:
pixel_values = feature_extractor(images=[img], return_tensors="pt").pixel_values
pixel_values = pixel_values.to(device)

In [26]:
max_length = 128
num_beams = 4

 # get model prediction
output_ids = model.generate(pixel_values, num_beams=num_beams, max_length=max_length)

 # decode the generated prediction
preds = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(preds)

TypeError: The current model class (DetrForObjectDetection) is not compatible with `.generate()`, as it doesn't have a language model head.

In [6]:
import torch
import transformers

# Load the pretrained model and tokenizer
model_name = 't5-small'
tokenizer = transformers.T5Tokenizer.from_pretrained(model_name)
model = transformers.T5ForConditionalGeneration.from_pretrained(model_name)

# Set the device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the function to generate captions
def generate_caption(words):
    # Encode the input words
    input_ids = tokenizer.encode(words, return_tensors='pt').to(device)

    # Generate the captions
    outputs = model.generate(input_ids=input_ids,
                             max_length=50,
                             num_beams=5,
                             early_stopping=True)
    
    # Decode the output captions
    captions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    
    return captions


In [20]:
captions = generate_caption(objects_detected[0])
print(captions)
# type(objects_detected)

['toothbrush toothbrush toothbrush']
